In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import zipfile
import pandas_profiling


In [4]:
zip_train = zipfile.ZipFile("/kaggle/input/sf-crime/train.csv.zip")
zip_test = zipfile.ZipFile("/kaggle/input/sf-crime/test.csv.zip")
f_train = zip_train.open("train.csv")
f_test = zip_test.open("test.csv")

In [5]:
train = pd.read_csv(f_train)
test = pd.read_csv(f_test)

NameError: name 'pd' is not defined

## Data descriptions (watch out only in train!!)

**Data fields**
1. Dates - timestamp of the crime incident
2. Category - category of the crime incident (only in train.csv). This is the target variable you are going to predict.
3. Descript - detailed description of the crime incident (only in train.csv)
4. DayOfWeek - the day of the week
5. PdDistrict - name of the Police Department District
6. Resolution - how the crime incident was resolved (only in train.csv)
7. Address - the approximate street address of the crime incident 
8. X - Longitude
9. Y - Latitude

## EDA Keypoint

1. coordinates
2. timestamps
3. address

In [11]:
train.profile_report()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
train.info()

In [ ]:
test.info()

In [92]:
test['PdDistrict'].unique()

array(['BAYVIEW', 'NORTHERN', 'INGLESIDE', 'TARAVAL', 'MISSION',
       'CENTRAL', 'TENDERLOIN', 'RICHMOND', 'SOUTHERN', 'PARK'],
      dtype=object)

## duplicate 제거

In [5]:
train.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)

In [6]:
## 공간좌표분석 library
import folium

In [7]:
mean_y = train['Y'].mean()
mean_x = train['X'].mean()
center = [mean_y, mean_x]

In [7]:
m = folium.Map(location=center, zoom_start=12)
for idx, row in train.sample(n=1000, random_state=1, axis=0).iterrows() :
    folium.Circle(radius=5, location=[row["Y"],row["X"]], color='#FF0000').add_to(m)
m

In [8]:
# to datetime
train['Dates'] = pd.to_datetime(train['Dates'])

In [9]:
train['Dates'][5]

Timestamp('2015-05-13 23:30:00')

In [10]:
# make new features
train['hour'] = train['Dates'].dt.hour
train['month'] = train['Dates'].dt.month # San Francisco는 사계절 기후가 비슷함
train['year'] = train['Dates'].dt.year

# begin = 0, mid = 1, end = 2
def transform_day(date:int) -> int:
    if date < 10:
        return 0
    elif date < 20:
        return 1
    else:
        return 2
train['day'] = train['Dates'].dt.day.apply(transform_day) 

In [11]:
train['Address_head'] = train['Address'].apply(lambda x: x.split('/')[0])
train['Address_tail'] = train['Address'].apply(lambda x: x.split('/')[1] if len(x.split('/'))>1 else -1)

In [38]:
train.head(1)

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,hour,month,day,year,Address_split,Address_head,Address_tail
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,23,5,1,2015,"[OAK ST , LAGUNA ST]",OAK ST,LAGUNA ST


In [20]:
train['Address_head'].value_counts()

800 Block of BRYANT ST             26245
MISSION ST                         11164
MARKET ST                           8060
800 Block of MARKET ST              6576
3RD ST                              6263
                                   ...  
0 Block of PALOALTO AV                 1
0 Block of TRINITY ST                  1
GAISER CT                              1
BRETHARTE TR                           1
1500 Block of JUNIPERO SERRA BL        1
Name: Address_head, Length: 12552, dtype: int64

In [43]:
train['Address_tail'].value_counts()

-1             615322
 MISSION ST     11246
 MARKET ST       7304
 JONES ST        4860
 LARKIN ST       4429
                ...  
 BERTITA ST         1
 GARCIA AV          1
 CRESCIO CT         1
 WALTHAM ST         1
 ZENO PL            1
Name: Address_tail, Length: 1695, dtype: int64

In [22]:
# 좌표를 100개의 grid로 나누기
min_x, max_x, min_y, max_y = min(train['X']), max(train['X']),min(train['Y']), max(train['Y'])
print(min_x, max_x, '\n', min_y, max_y)

-122.51364206429 -120.5 
 37.7078790224135 90.0


In [39]:
print(len(train['X'].unique()))
print(len(train['Y'].unique()))

34243
34243


In [85]:
print("X좌표 3번째 자리까지 round ->", len((train['X'].apply(lambda x: np.round(x, 3))).unique()))
print("X좌표 4번째 자리까지 round ->", len((train['X'].apply(lambda x: np.round(x, 4))).unique()))
print("Y좌표 3번째 자리까지 round ->", len((train['Y'].apply(lambda x: np.round(x, 3))).unique()))
print("Y좌표 4번째 자리까지 round ->", len((train['Y'].apply(lambda x: np.round(x, 4))).unique()))

X좌표 3번째 자리까지 round -> 147
X좌표 4번째 자리까지 round -> 1387
Y좌표 3번째 자리까지 round -> 107
Y좌표 4번째 자리까지 round -> 1018


In [ ]:
'''
X
최저 : -122.51364206429
최고 : -122.364937 

Y
최저 : 37.7078790224135
최고 : 37.819975
'''

In [63]:
x_min, x_max = -122.51364206429, -122.364937
y_min, y_max = 37.7078790224135, 37.819975
rate = (x_max-x_min)/(y_max-y_min)
x_list = np.linspace(-122.513642, -122.364937, 1000*rate)
y_list = np.linspace(37.707879, 37.819975, 1000)

1.3265869792272467

In [78]:
# BS로 효율화
import bisect
train['new_X'] = None
train['new_Y'] = None

for idx, row in train[["X", "Y"]].iterrows():
    if idx == 5000:
        break
    train['new_X'] = bisect.bisect_left(x_list, row["X"])
    train['new_Y'] = bisect.bisect_left(y_list, row["Y"])
print(len(train['new_X'].unique()))
print(len(train['new_Y'].unique()))

-122.513642

## 좌표 분포가 조밀해서 유의미한 grid 생성 어려움 -> round값으로

In [86]:
train.drop(columns=['Dates', 'Resolution', 'Address_head', 'Address_tail'])

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,hour,month,year,day,Address_head,Address_tail,new_X,new_Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,23,5,2015,1,OAK ST,LAGUNA ST,937,679
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,23,5,2015,1,OAK ST,LAGUNA ST,937,679
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414,23,5,2015,1,VANNESS AV,GREENWICH ST,937,679
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873,23,5,2015,1,1500 Block of LOMBARD ST,-1,937,679
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541,23,5,2015,1,100 Block of BRODERICK ST,-1,937,679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
878044,2003-01-06 00:15:00,ROBBERY,ROBBERY ON THE STREET WITH A GUN,Monday,TARAVAL,NONE,FARALLONES ST / CAPITOL AV,-122.459033,37.714056,0,1,2003,0,FARALLONES ST,CAPITOL AV,937,679
878045,2003-01-06 00:01:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Monday,INGLESIDE,NONE,600 Block of EDNA ST,-122.447364,37.731948,0,1,2003,0,600 Block of EDNA ST,-1,937,679
878046,2003-01-06 00:01:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Monday,SOUTHERN,NONE,5TH ST / FOLSOM ST,-122.403390,37.780266,0,1,2003,0,5TH ST,FOLSOM ST,937,679
878047,2003-01-06 00:01:00,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",Monday,SOUTHERN,NONE,TOWNSEND ST / 2ND ST,-122.390531,37.780607,0,1,2003,0,TOWNSEND ST,2ND ST,937,679


In [6]:
enc = OneHotEncoder(sparse=False, handle_unknown='ignore')
cols = train['PdDistrict'].unique()
pdict = dict()

for value in cols:
    pdict[value] = len(pdict) + 1

NameError: name 'OneHotEncoder' is not defined

{'NORTHERN': 0,
 'PARK': 1,
 'INGLESIDE': 2,
 'BAYVIEW': 3,
 'RICHMOND': 4,
 'CENTRAL': 5,
 'TARAVAL': 6,
 'TENDERLOIN': 7,
 'MISSION': 8,
 'SOUTHERN': 9}

In [169]:
def preprocess(input_data:pd.DataFrame, test=False):
    global enc
    global pdict
    data = input_data.copy()
    if not test:
        data.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)
        data.drop(columns=['Resolution', 'Descript'], inplace=True, axis=1)
    data['Dates'] = pd.to_datetime(data['Dates'])
    # Date feature engineering
    # make new features
    data['hour'] = data['Dates'].dt.hour
    data['month'] = data['Dates'].dt.month # San Francisco는 사계절 기후가 비슷함
    data['year'] = data['Dates'].dt.year

    # begin = 1, mid = 2, end = 3
    def transform_day(date:int) -> int:
        if date < 10:
            return 1
        elif date < 20:
            return 2
        else:
            return 3
    data['day'] = data['Dates'].dt.day.apply(transform_day) 
    data.drop('Dates', inplace=True, axis=1)
    day_of_week = {"Monday":1, "Tuesday":2, "Wednesday":3, "Thursday":4, "Friday":5, "Saturday":6, "Sunday":7}
    data['DayOfWeek'] = data['DayOfWeek'].apply(lambda x: day_of_week.get(x, 8))
    
    # Coordinates feature engineering
    data['X'] = data['X'].apply(lambda x: np.round(x, 4))
    data['Y'] = data['Y'].apply(lambda x: np.round(x, 4))
    data.drop('Address', inplace=True, axis=1)
    
    # PdDistrict encoding
    data['PdDistrict'] = data['PdDistrict'].apply(lambda x: pdict.get(x, -1))
    
    # One-hot encoding
    if not test:
        y = enc.fit_transform(data['Category'].values.reshape(-1,1))
        X = data.drop('Category', axis=1)
        return X, y
    return data

In [170]:
X, y = preprocess(train)

In [171]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 875726 entries, 0 to 878048
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   DayOfWeek   875726 non-null  int64  
 1   PdDistrict  875726 non-null  int64  
 2   X           875726 non-null  float64
 3   Y           875726 non-null  float64
 4   hour        875726 non-null  int64  
 5   month       875726 non-null  int64  
 6   year        875726 non-null  int64  
 7   day         875726 non-null  int64  
dtypes: float64(2), int64(6)
memory usage: 60.1 MB


In [159]:
y

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [36]:
print((train['Y'] == 90.0).sum())
train['Y'].sort_values()[-70::]

67


89376     37.819975
288672    37.819975
354212    37.819975
677260    90.000000
783919    90.000000
            ...    
788574    90.000000
661106    90.000000
767318    90.000000
686997    90.000000
660712    90.000000
Name: Y, Length: 70, dtype: float64

In [61]:
print((train["X"] > -122).sum())
train['X'].sort_values()[-70::]

67


772670   -122.364937
804125   -122.364937
774410   -122.364937
719993   -120.500000
712309   -120.500000
             ...    
666430   -120.500000
767502   -120.500000
767501   -120.500000
727080   -120.500000
667042   -120.500000
Name: X, Length: 70, dtype: float64

In [ ]:
# latitude == 90은 outlier로 간주

In [ ]:
# 좌표 데이터에 대한 KMeans

In [50]:
enc = LabelEncoder()
train["Category"] = enc.fit_transform(train["Category"])
points = train[["X", "Y", "Category"]]
kmeans = KMeans(n_clusters=100)
kmeans.fit(points)

# 결과 확인
result_by_sklearn = points.copy()
result_by_sklearn["cluster"] = kmeans.labels_
result_by_sklearn.head()

,X,Y,Category,cluster
0,-122.425892,37.774599,37,8
1,-122.425892,37.774599,21,92
2,-122.424363,37.800414,21,92
3,-122.426995,37.800873,16,90
4,-122.438738,37.771541,16,68


In [19]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 875726 entries, 0 to 878048
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Dates         875726 non-null  datetime64[ns]
 1   Category      875726 non-null  object        
 2   Descript      875726 non-null  object        
 3   DayOfWeek     875726 non-null  object        
 4   PdDistrict    875726 non-null  object        
 5   Resolution    875726 non-null  object        
 6   Address       875726 non-null  object        
 7   X             875726 non-null  float64       
 8   Y             875726 non-null  float64       
 9   hour          875726 non-null  int64         
 10  month         875726 non-null  int64         
 11  year          875726 non-null  int64         
 12  day           875726 non-null  int64         
 13  Address_head  875726 non-null  object        
 14  Address_tail  875726 non-null  object        
dtypes: datetime64[ns]

## Model : XGBoost

In [162]:
model = XGBClassifier(n_estimators=500, learning_rate=0.05, max_depth=5, random_state = 32,tree_method='gpu_hist', gpu_id=0)

In [165]:
!nvidia-smi

Tue Oct 25 18:05:27 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    34W / 250W |    975MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [172]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

In [173]:
model.fit(X_train,y_train)
pred = model.predict(X_test)
loss = log_loss(y_test, pred)
print(loss)

4.05138388094979


In [ ]:
forest.fit(X_train,y_train)
pred = forest.predict(X_test)
loss = log_loss(y_test, pred)
print(loss)

In [181]:
from sklearn.ensemble import RandomForestClassifier

In [182]:
forest = RandomForestClassifier(n_jobs=-1)

In [179]:
y_test.shape

(87573, 39)

In [180]:
pred.shape

(87573, 39)